In [1]:
import pickle
from glob import iglob
import numpy as np
import librosa
from shutil import rmtree
from constants import *
import speech_recognition as speech_r

DATA_AUDIO_DIR = '../augmentation3_train_data'
DATA_RESAMPLED_AUDIO_DIR = '../resampled_augmentation3_1_train_data'

list_dir = os.listdir(DATA_AUDIO_DIR)
list_dir.sort()

r = speech_r.Recognizer() 

OUTPUT_DIR = '../output_una_re_augmentation3_1'
OUTPUT_DIR_TRAIN = os.path.join(OUTPUT_DIR, 'train')
OUTPUT_DIR_TEST = os.path.join(OUTPUT_DIR, 'test')
OUTPUT_DIR_RESAMPLED_TRAIN = os.path.join(OUTPUT_DIR, 'resampled_train')
OUTPUT_DIR_RESAMPLED_TEST = os.path.join(OUTPUT_DIR, 'resampled_test')

PAD_SIZE = 0
TARGET_LENGTH = 200000
TARGET_SR = 8000
MAX_AUDIO_LENGTH = 200
AUDIO_LENGTH = TARGET_LENGTH
SEGMENT_NUM = 1

In [2]:
def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise


def del_folder(path):
    try:
        rmtree(path)
    except:
        pass

### Delete pickle file & Make directory

In [3]:
del_folder(OUTPUT_DIR_TRAIN)
del_folder(OUTPUT_DIR_TEST)
mkdir_p(OUTPUT_DIR_TRAIN)
mkdir_p(OUTPUT_DIR_TEST)

In [4]:
del_folder(OUTPUT_DIR_RESAMPLED_TRAIN)
del_folder(OUTPUT_DIR_RESAMPLED_TEST)
mkdir_p(OUTPUT_DIR_RESAMPLED_TRAIN)
mkdir_p(OUTPUT_DIR_RESAMPLED_TEST)

In [5]:
del_folder(DATA_RESAMPLED_AUDIO_DIR)
mkdir_p(DATA_RESAMPLED_AUDIO_DIR)

## Audio Resampling 

음성 길이를 TARGET_LENGTH로 맞추기 위한 전처리 진행   
Default Samplerate는 TARGET_SR 이며, TARGET_SR로 Audio를 load 하여 Audio Duration을 측정한다.  

**Audio Duration X Sample Rate = Result Audio Length**   

Audio Duration과 Result Audio Length(TARGET_LENGTH)를 통해 Sample Rate를 구한다.  


In [6]:
def resampling_audio():
     for i, wav_filename in enumerate(iglob(os.path.join(DATA_AUDIO_DIR, '**/**.wav'), recursive=True)):
        print(wav_filename)
        y, sr = librosa.load(wav_filename, sr=TARGET_SR, mono=True, duration=MAX_AUDIO_LENGTH)
        audio_duration = len(y) / sr
        
        resample_sr = TARGET_LENGTH / audio_duration        
        resample = librosa.resample(y, sr, resample_sr)
        print(i, wav_filename)
        print("Default SR : {}, Audio Length(Default SR) : {}, Audio Duration : {}".format(TARGET_SR, len(y), audio_duration))
        print("Audio Duration : {}, Resampling SR : {}, Result Audio Length : {}".format(audio_duration, resample_sr, len(resample)))
        
        if len(resample) < TARGET_LENGTH:
            resample = np.concatenate((resample, np.zeros(shape=(TARGET_LENGTH - len(resample), 1))))
        resample = resample[:TARGET_LENGTH]
        
#         print("Final Audio Length : {}".format(len(resample)))

        
        output_folder = DATA_RESAMPLED_AUDIO_DIR
        if not os.path.exists(os.path.join(DATA_RESAMPLED_AUDIO_DIR, wav_filename.split('/')[2])):
                mkdir_p(os.path.join(DATA_RESAMPLED_AUDIO_DIR, wav_filename.split('/')[2]))
                
        output_filename = os.path.join(DATA_RESAMPLED_AUDIO_DIR, wav_filename.split('/')[2], str(wav_filename.replace('.','/').split('/')[5])+".pkl")
        print("Output File Name: {}\n".format(output_filename))
        
        result = {'resampled_audio': resample,
               'resample_sr': resample_sr}
        
#         np.savetxt( DATA_RESAMPLED_AUDIO_DIR+"/"+str(wav_filename.replace('.','/').split('/')[5])+".txt", resample, delimiter=',')
        
        with open(output_filename, 'wb') as w:
                  pickle.dump(result, w)


In [7]:
resampling_audio()

../augmentation3_train_data/2017019770040_kimdajeong/5.wav
0 ../augmentation3_train_data/2017019770040_kimdajeong/5.wav
Default SR : 8000, Audio Length(Default SR) : 201088, Audio Duration : 25.136
Audio Duration : 25.136, Resampling SR : 7956.7154678548695, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019770040_kimdajeong/5.pkl

../augmentation3_train_data/2017019770040_kimdajeong/4.wav
1 ../augmentation3_train_data/2017019770040_kimdajeong/4.wav
Default SR : 8000, Audio Length(Default SR) : 195712, Audio Duration : 24.464
Audio Duration : 24.464, Resampling SR : 8175.277959450622, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019770040_kimdajeong/4.pkl

../augmentation3_train_data/2017019770040_kimdajeong/8.wav
2 ../augmentation3_train_data/2017019770040_kimdajeong/8.wav
Default SR : 8000, Audio Length(Default SR) : 298514, Audio Duration : 37.31425
Audio Duration : 37.31425, Resampling SR : 53

23 ../augmentation3_train_data/2017019740042_shinbeom/6.wav
Default SR : 8000, Audio Length(Default SR) : 176780, Audio Duration : 22.0975
Audio Duration : 22.0975, Resampling SR : 9050.797601538636, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019740042_shinbeom/6.pkl

../augmentation3_train_data/2017019740042_shinbeom/7.wav
24 ../augmentation3_train_data/2017019740042_shinbeom/7.wav
Default SR : 8000, Audio Length(Default SR) : 170954, Audio Duration : 21.36925
Audio Duration : 21.36925, Resampling SR : 9359.242837254465, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019740042_shinbeom/7.pkl

../augmentation3_train_data/2017019740042_shinbeom/3.wav
25 ../augmentation3_train_data/2017019740042_shinbeom/3.wav
Default SR : 8000, Audio Length(Default SR) : 254976, Audio Duration : 31.872
Audio Duration : 31.872, Resampling SR : 6275.100401606425, Result Audio Length : 200000
Output File Name: ../re

45 ../augmentation3_train_data/2017019880008_jangsoojin/3.wav
Default SR : 8000, Audio Length(Default SR) : 222079, Audio Duration : 27.759875
Audio Duration : 27.759875, Resampling SR : 7204.643392666573, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019880008_jangsoojin/3.pkl

../augmentation3_train_data/2017019880008_jangsoojin/2.wav
46 ../augmentation3_train_data/2017019880008_jangsoojin/2.wav
Default SR : 8000, Audio Length(Default SR) : 223669, Audio Duration : 27.958625
Audio Duration : 27.958625, Resampling SR : 7153.427609548037, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019880008_jangsoojin/2.pkl

../augmentation3_train_data/2017019880008_jangsoojin/10.wav
47 ../augmentation3_train_data/2017019880008_jangsoojin/10.wav
Default SR : 8000, Audio Length(Default SR) : 169751, Audio Duration : 21.218875
Audio Duration : 21.218875, Resampling SR : 9425.570394283392, Result Audio Length : 20

68 ../augmentation3_train_data/2017019770029_jueunhong/1.wav
Default SR : 8000, Audio Length(Default SR) : 240064, Audio Duration : 30.008
Audio Duration : 30.008, Resampling SR : 6664.889362836577, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019770029_jueunhong/1.pkl

../augmentation3_train_data/2017019770029_jueunhong/9.wav
69 ../augmentation3_train_data/2017019770029_jueunhong/9.wav
Default SR : 8000, Audio Length(Default SR) : 183040, Audio Duration : 22.88
Audio Duration : 22.88, Resampling SR : 8741.258741258742, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019770029_jueunhong/9.pkl

../augmentation3_train_data/2017019740021_kwakbokyeong/5.wav
70 ../augmentation3_train_data/2017019740021_kwakbokyeong/5.wav
Default SR : 8000, Audio Length(Default SR) : 330923, Audio Duration : 41.365375
Audio Duration : 41.365375, Resampling SR : 4834.961607382987, Result Audio Length : 200000
Output File 

90 ../augmentation3_train_data/2017019880002_kimkihyeon/5.wav
Default SR : 8000, Audio Length(Default SR) : 190682, Audio Duration : 23.83525
Audio Duration : 23.83525, Resampling SR : 8390.933596249253, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019880002_kimkihyeon/5.pkl

../augmentation3_train_data/2017019880002_kimkihyeon/4.wav
91 ../augmentation3_train_data/2017019880002_kimkihyeon/4.wav
Default SR : 8000, Audio Length(Default SR) : 210045, Audio Duration : 26.255625
Audio Duration : 26.255625, Resampling SR : 7617.41531576567, Result Audio Length : 200001
Output File Name: ../resampled_augmentation3_1_train_data/2017019880002_kimkihyeon/4.pkl

../augmentation3_train_data/2017019880002_kimkihyeon/8.wav
92 ../augmentation3_train_data/2017019880002_kimkihyeon/8.wav
Default SR : 8000, Audio Length(Default SR) : 292776, Audio Duration : 36.597
Audio Duration : 36.597, Resampling SR : 5464.928819302128, Result Audio Length : 200000
Output

...

1183 ../augmentation3_train_data/2017019740014_leesumin/6.wav
Default SR : 8000, Audio Length(Default SR) : 297860, Audio Duration : 37.2325
Audio Duration : 37.2325, Resampling SR : 5371.651111260323, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019740014_leesumin/6.pkl

../augmentation3_train_data/2017019740014_leesumin/7.wav
1184 ../augmentation3_train_data/2017019740014_leesumin/7.wav
Default SR : 8000, Audio Length(Default SR) : 176132, Audio Duration : 22.0165
Audio Duration : 22.0165, Resampling SR : 9084.09601889492, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019740014_leesumin/7.pkl

../augmentation3_train_data/2017019740014_leesumin/3.wav
1185 ../augmentation3_train_data/2017019740014_leesumin/3.wav
Default SR : 8000, Audio Length(Default SR) : 239875, Audio Duration : 29.984375
Audio Duration : 29.984375, Resampling SR : 6670.140698280355, Result Audio Length : 200000
Output File Na

1205 ../augmentation3_train_data/2017019740032_choiyeongmi/3.wav
Default SR : 8000, Audio Length(Default SR) : 216982, Audio Duration : 27.12275
Audio Duration : 27.12275, Resampling SR : 7373.8835479440695, Result Audio Length : 200001
Output File Name: ../resampled_augmentation3_1_train_data/2017019740032_choiyeongmi/3.pkl

../augmentation3_train_data/2017019740032_choiyeongmi/2.wav
1206 ../augmentation3_train_data/2017019740032_choiyeongmi/2.wav
Default SR : 8000, Audio Length(Default SR) : 219030, Audio Duration : 27.37875
Audio Duration : 27.37875, Resampling SR : 7304.935396977583, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019740032_choiyeongmi/2.pkl

../augmentation3_train_data/2017019740032_choiyeongmi/10.wav
1207 ../augmentation3_train_data/2017019740032_choiyeongmi/10.wav
Default SR : 8000, Audio Length(Default SR) : 181325, Audio Duration : 22.665625
Audio Duration : 22.665625, Resampling SR : 8823.93492347994, Result Audio Le

1227 ../augmentation3_train_data/2017019740011_kimhyeryeong/10.wav
Default SR : 8000, Audio Length(Default SR) : 173243, Audio Duration : 21.655375
Audio Duration : 21.655375, Resampling SR : 9235.582390053278, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019740011_kimhyeryeong/10.pkl

../augmentation3_train_data/2017019740011_kimhyeryeong/1.wav
1228 ../augmentation3_train_data/2017019740011_kimhyeryeong/1.wav
Default SR : 8000, Audio Length(Default SR) : 251584, Audio Duration : 31.448
Audio Duration : 31.448, Resampling SR : 6359.70490969219, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019740011_kimhyeryeong/1.pkl

../augmentation3_train_data/2017019740011_kimhyeryeong/9.wav
1229 ../augmentation3_train_data/2017019740011_kimhyeryeong/9.wav
Default SR : 8000, Audio Length(Default SR) : 176394, Audio Duration : 22.04925
Audio Duration : 22.04925, Resampling SR : 9070.603308502556, Result Audio 

1250 ../augmentation3_train_data/2017019880034_leesujung/5.wav
Default SR : 8000, Audio Length(Default SR) : 235040, Audio Duration : 29.38
Audio Duration : 29.38, Resampling SR : 6807.351940095303, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019880034_leesujung/5.pkl

../augmentation3_train_data/2017019880034_leesujung/4.wav
1251 ../augmentation3_train_data/2017019880034_leesujung/4.wav
Default SR : 8000, Audio Length(Default SR) : 241448, Audio Duration : 30.181
Audio Duration : 30.181, Resampling SR : 6626.685663165567, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019880034_leesujung/4.pkl

../augmentation3_train_data/2017019880034_leesujung/8.wav
1252 ../augmentation3_train_data/2017019880034_leesujung/8.wav
Default SR : 8000, Audio Length(Default SR) : 185231, Audio Duration : 23.153875
Audio Duration : 23.153875, Resampling SR : 8637.862992695607, Result Audio Length : 200000
Output File 

1273 ../augmentation3_train_data/2017019880003_kimminji/6.wav
Default SR : 8000, Audio Length(Default SR) : 171231, Audio Duration : 21.403875
Audio Duration : 21.403875, Resampling SR : 9344.102411362428, Result Audio Length : 200000
Output File Name: ../resampled_augmentation3_1_train_data/2017019880003_kimminji/6.pkl

../augmentation3_train_data/2017019880003_kimminji/7.wav
1274 ../augmentation3_train_data/2017019880003_kimminji/7.wav
Default SR : 8000, Audio Length(Default SR) : 169801, Audio Duration : 21.225125
Audio Duration : 21.225125, Resampling SR : 9422.794918757842, Result Audio Length : 200001
Output File Name: ../resampled_augmentation3_1_train_data/2017019880003_kimminji/7.pkl

../augmentation3_train_data/2017019880003_kimminji/3.wav
1275 ../augmentation3_train_data/2017019880003_kimminji/3.wav
Default SR : 8000, Audio Length(Default SR) : 234658, Audio Duration : 29.33225
Audio Duration : 29.33225, Resampling SR : 6818.433635333124, Result Audio Length : 200001
Output 

In [8]:
order = 0
list_dir.sort()
class_ids = {list_dir[i]: i for i in range(len(list_dir))}

def extract_class_id(wav_filename):
    return class_ids.get(wav_filename.split('/')[2])

def read_audio_from_filename(filename, target_sr):
    audio, _ = librosa.load(filename, sr=TARGET_SR, mono=True)
    audio = audio.reshape(-1, 1)
    return audio

def get_audio_buf(filename):
    audio_buf = None
    sample_rate = None
    
    with open(filename, 'rb') as f:
        audio_element = pickle.load(f)
        audio_buf = audio_element['resampled_audio']
        sample_rate = audio_element['resample_sr']
    
    return audio_buf, sample_rate

def convert_data():
    
    for i, wav_filename in enumerate(iglob(os.path.join(DATA_RESAMPLED_AUDIO_DIR, '**/**.pkl'), recursive=True)):
        class_id = extract_class_id(wav_filename)
#         audio_buf = read_audio_from_filename(wav_filename, target_sr=TARGET_SR)
        audio_buf,_ = get_audio_buf(wav_filename)
        print(type(audio_buf))
        
        # normalize mean 0, variance 1
        audio_buf = (audio_buf - np.mean(audio_buf)) / np.std(audio_buf)
        original_length = len(audio_buf)
        print(i, wav_filename, original_length, np.round(np.mean(audio_buf), 4), np.std(audio_buf))
        
        voice_seg = []
        
        if original_length >= AUDIO_LENGTH + PAD_SIZE:
            audio_seg = audio_buf[PAD_SIZE : AUDIO_LENGTH + PAD_SIZE]
            voice_seg.append(audio_seg)
            print("Audio Segment Length : {}".format(len(audio_seg)))
        else:
            print('Drop Audio Segment, Audio length={}'.format(len(audio_buf)))
            
        output_folder = OUTPUT_DIR_RESAMPLED_TRAIN
        if wav_filename[-5:] == '5.pkl' or wav_filename[-5:] == '6.pkl':
            output_folder = OUTPUT_DIR_RESAMPLED_TEST
            
        print(str(wav_filename[24:-4]).replace('/', '_') )
        output_filename = os.path.join(output_folder ,wav_filename.split('/')[2]+"_"+str(wav_filename.replace('.','/').split('/')[5]) + '.pkl')
#         output_filename = os.path.join(output_folder, str(wav_filename[24:-4]).replace('/', '_') + '.pkl')

        out_segs = []
        for i_seg, audio_seg in enumerate(voice_seg) :
            out = {'class_id': class_id,
               'audio': audio_seg,
               'sr': TARGET_SR}
            
            with open(output_filename, 'wb') as w:
                pickle.dump(out, w)

            print("Output File Name : "+output_filename)
        
        print("-----------------")
      

### Convert Data

In [9]:
convert_data()

<class 'numpy.ndarray'>
0 ../resampled_augmentation3_1_train_data/2017019770040_kimdajeong/8.pkl 200000 -0.0 1.0000001
Audio Segment Length : 200000
n3_1_train_data_2017019770040_kimdajeong_8
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019770040_kimdajeong_8.pkl
-----------------
<class 'numpy.ndarray'>
1 ../resampled_augmentation3_1_train_data/2017019770040_kimdajeong/5.pkl 200000 -0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019770040_kimdajeong_5
Output File Name : ../output_una_re_augmentation3_1/resampled_test/2017019770040_kimdajeong_5.pkl
-----------------
<class 'numpy.ndarray'>
2 ../resampled_augmentation3_1_train_data/2017019770040_kimdajeong/7.pkl 200000 -0.0 0.99999994
Audio Segment Length : 200000
n3_1_train_data_2017019770040_kimdajeong_7
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019770040_kimdajeong_7.pkl
-----------------
<class 'numpy.ndarray'>
3 ../resampled_augmentation3_1_train_data/2017019770040

Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019880002_kimkihyeon_10.pkl
-----------------
<class 'numpy.ndarray'>
97 ../resampled_augmentation3_1_train_data/2017019880002_kimkihyeon/9.pkl 200000 -0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019880002_kimkihyeon_9
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019880002_kimkihyeon_9.pkl
-----------------
<class 'numpy.ndarray'>
98 ../resampled_augmentation3_1_train_data/2017019880002_kimkihyeon/3.pkl 200000 -0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019880002_kimkihyeon_3
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019880002_kimkihyeon_3.pkl
-----------------
<class 'numpy.ndarray'>
99 ../resampled_augmentation3_1_train_data/2017019880002_kimkihyeon/2.pkl 200000 -0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019880002_kimkihyeon_2
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019880002_kimki

Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740006_kangsubin_1.pkl
-----------------
<class 'numpy.ndarray'>
214 ../resampled_augmentation3_1_train_data/2017019740006_kangsubin/4.pkl 200000 0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019740006_kangsubin_4
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740006_kangsubin_4.pkl
-----------------
<class 'numpy.ndarray'>
215 ../resampled_augmentation3_1_train_data/2017019740006_kangsubin/6.pkl 200000 0.0 0.9999999
Audio Segment Length : 200000
n3_1_train_data_2017019740006_kangsubin_6
Output File Name : ../output_una_re_augmentation3_1/resampled_test/2017019740006_kangsubin_6.pkl
-----------------
<class 'numpy.ndarray'>
216 ../resampled_augmentation3_1_train_data/2017019740006_kangsubin/10.pkl 200000 0.0 0.9999999
Audio Segment Length : 200000
n3_1_train_data_2017019740006_kangsubin_10
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740006_ka

334 ../resampled_augmentation3_1_train_data/2017019770016_parkjongae/4.pkl 200000 0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019770016_parkjongae_4
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019770016_parkjongae_4.pkl
-----------------
<class 'numpy.ndarray'>
335 ../resampled_augmentation3_1_train_data/2017019770016_parkjongae/6.pkl 200000 -0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019770016_parkjongae_6
Output File Name : ../output_una_re_augmentation3_1/resampled_test/2017019770016_parkjongae_6.pkl
-----------------
<class 'numpy.ndarray'>
336 ../resampled_augmentation3_1_train_data/2017019770016_parkjongae/10.pkl 200000 0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019770016_parkjongae_10
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019770016_parkjongae_10.pkl
-----------------
<class 'numpy.ndarray'>
337 ../resampled_augmentation3_1_train_data/2017019770016_parkjongae/9.pkl 200000 0.0

Output File Name : ../output_una_re_augmentation3_1/resampled_test/2017019880042_namyeji_6.pkl
-----------------
<class 'numpy.ndarray'>
456 ../resampled_augmentation3_1_train_data/2017019880042_namyeji/10.pkl 200000 -0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019880042_namyeji_10
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019880042_namyeji_10.pkl
-----------------
<class 'numpy.ndarray'>
457 ../resampled_augmentation3_1_train_data/2017019880042_namyeji/9.pkl 200000 -0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019880042_namyeji_9
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019880042_namyeji_9.pkl
-----------------
<class 'numpy.ndarray'>
458 ../resampled_augmentation3_1_train_data/2017019880042_namyeji/3.pkl 200000 0.0 0.99999994
Audio Segment Length : 200000
n3_1_train_data_2017019880042_namyeji_3
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019880042_namyeji_3.pkl
--------

...

696 ../resampled_augmentation3_1_train_data/2017019770037_jeonghojun/10.pkl 200000 -0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019770037_jeonghojun_10
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019770037_jeonghojun_10.pkl
-----------------
<class 'numpy.ndarray'>
697 ../resampled_augmentation3_1_train_data/2017019770037_jeonghojun/9.pkl 200000 -0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019770037_jeonghojun_9
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019770037_jeonghojun_9.pkl
-----------------
<class 'numpy.ndarray'>
698 ../resampled_augmentation3_1_train_data/2017019770037_jeonghojun/3.pkl 200000 -0.0 0.9999999
Audio Segment Length : 200000
n3_1_train_data_2017019770037_jeonghojun_3
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019770037_jeonghojun_3.pkl
-----------------
<class 'numpy.ndarray'>
699 ../resampled_augmentation3_1_train_data/2017019770037_jeonghojun/2.pkl 2

<class 'numpy.ndarray'>
817 ../resampled_augmentation3_1_train_data/2017019740043_kimtaeri/9.pkl 200000 0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019740043_kimtaeri_9
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740043_kimtaeri_9.pkl
-----------------
<class 'numpy.ndarray'>
818 ../resampled_augmentation3_1_train_data/2017019740043_kimtaeri/3.pkl 200000 -0.0 0.99999994
Audio Segment Length : 200000
n3_1_train_data_2017019740043_kimtaeri_3
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740043_kimtaeri_3.pkl
-----------------
<class 'numpy.ndarray'>
819 ../resampled_augmentation3_1_train_data/2017019740043_kimtaeri/2.pkl 200000 0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019740043_kimtaeri_2
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740043_kimtaeri_2.pkl
-----------------
<class 'numpy.ndarray'>
820 ../resampled_augmentation3_1_train_data/2017019740020_kimseongje/8.pkl

Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740017_kwakmihyang_2.pkl
-----------------
<class 'numpy.ndarray'>
940 ../resampled_augmentation3_1_train_data/2017019740034_kimdongwook/8.pkl 200000 -0.0 1.0000001
Audio Segment Length : 200000
n3_1_train_data_2017019740034_kimdongwook_8
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740034_kimdongwook_8.pkl
-----------------
<class 'numpy.ndarray'>
941 ../resampled_augmentation3_1_train_data/2017019740034_kimdongwook/5.pkl 200000 0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019740034_kimdongwook_5
Output File Name : ../output_una_re_augmentation3_1/resampled_test/2017019740034_kimdongwook_5.pkl
-----------------
<class 'numpy.ndarray'>
942 ../resampled_augmentation3_1_train_data/2017019740034_kimdongwook/7.pkl 200000 -0.0 1.0000001
Audio Segment Length : 200000
n3_1_train_data_2017019740034_kimdongwook_7
Output File Name : ../output_una_re_augmentation3_1/resampled_trai

Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740008_chochaeyeon_7.pkl
-----------------
<class 'numpy.ndarray'>
1063 ../resampled_augmentation3_1_train_data/2017019740008_chochaeyeon/1.pkl 200000 -0.0 0.9999999
Audio Segment Length : 200000
n3_1_train_data_2017019740008_chochaeyeon_1
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740008_chochaeyeon_1.pkl
-----------------
<class 'numpy.ndarray'>
1064 ../resampled_augmentation3_1_train_data/2017019740008_chochaeyeon/4.pkl 200000 -0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019740008_chochaeyeon_4
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740008_chochaeyeon_4.pkl
-----------------
<class 'numpy.ndarray'>
1065 ../resampled_augmentation3_1_train_data/2017019740008_chochaeyeon/6.pkl 200000 0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019740008_chochaeyeon_6
Output File Name : ../output_una_re_augmentation3_1/resampled_test/2

<class 'numpy.ndarray'>
1184 ../resampled_augmentation3_1_train_data/2017019740014_leesumin/4.pkl 200000 0.0 1.0000001
Audio Segment Length : 200000
n3_1_train_data_2017019740014_leesumin_4
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740014_leesumin_4.pkl
-----------------
<class 'numpy.ndarray'>
1185 ../resampled_augmentation3_1_train_data/2017019740014_leesumin/6.pkl 200000 0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019740014_leesumin_6
Output File Name : ../output_una_re_augmentation3_1/resampled_test/2017019740014_leesumin_6.pkl
-----------------
<class 'numpy.ndarray'>
1186 ../resampled_augmentation3_1_train_data/2017019740014_leesumin/10.pkl 200000 0.0 1.0
Audio Segment Length : 200000
n3_1_train_data_2017019740014_leesumin_10
Output File Name : ../output_una_re_augmentation3_1/resampled_train/2017019740014_leesumin_10.pkl
-----------------
<class 'numpy.ndarray'>
1187 ../resampled_augmentation3_1_train_data/2017019740014_leesumin/9.p

In [10]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Lambda, Bidirectional, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow import convert_to_tensor
from tensorflow import expand_dims
import numpy as np
import pickle
import os
from glob import glob
import tensorflow

print(tensorflow.__version__)

2.1.0


In [11]:
list_dir = os.listdir(DATA_RESAMPLED_AUDIO_DIR)
list_dir.sort()

class_ids = {list_dir[i]: i for i in range(len(list_dir))}

In [12]:
def m5(num_classes):
    print("Class Num", num_classes)
    print('Using Model M5')
    m = Sequential()
    m.add(Conv1D(128,
                 input_shape=[AUDIO_LENGTH, 1],
                 kernel_size=80,
                 strides=4,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(128,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(256,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(512,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Lambda(lambda x: K.mean(x, axis=1)))  # Same as GAP for 1D Conv Layer
    m.add(Dense(num_classes, activation='softmax'))
    return m

In [13]:
def get_data(file_list):
    def load_into(_filename, _x, _y):
        with open(_filename, 'rb') as f:
            audio_element = pickle.load(f)
            _x.append(audio_element['audio'])
            _y.append(int(audio_element['class_id']))

    x, y = [], []
    for filename in file_list:
        load_into(filename, x, y)
    return np.array(x), np.array(y)

In [14]:
num_classes = len(list_dir)
model = m5(num_classes)

if model is None:
    exit('Something went wrong!!')

model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
print(model.summary())

Class Num 128
Using Model M5
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 50000, 128)        10368     
_________________________________________________________________
batch_normalization (BatchNo (None, 50000, 128)        512       
_________________________________________________________________
activation (Activation)      (None, 50000, 128)        0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 12500, 128)        0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 12500, 128)        49280     
_________________________________________________________________
batch_normalization_1 (Batch (None, 12500, 128)        512       
_________________________________________________________________
activation_1 (Activation)  

In [15]:
train_files = glob(os.path.join(OUTPUT_DIR_RESAMPLED_TRAIN, '**.pkl'))
print(os.path.join(OUTPUT_DIR_RESAMPLED_TRAIN, '**.pkl'))
x_tr, y_tr = get_data(train_files)

y_tr = to_categorical(y_tr, num_classes=num_classes)
x_tr = x_tr.reshape(-1, x_tr.shape[1], 1)


test_files = glob(os.path.join(OUTPUT_DIR_RESAMPLED_TEST, '**.pkl'))
x_te, y_te = get_data(test_files)

y_te = to_categorical(y_te, num_classes=num_classes)
x_te = x_te.reshape(-1, x_te.shape[1], 1)


print('x_tr.shape =', x_tr.shape)
print('y_tr.shape =', y_tr.shape)
print('x_te.shape =', x_te.shape)
print('y_te.shape =', y_te.shape)

../output_una_re_augmentation3_1/resampled_train/**.pkl
x_tr.shape = (1024, 200000, 1)
y_tr.shape = (1024, 128)
x_te.shape = (256, 200000, 1)
y_te.shape = (256, 128)


In [16]:
# if the accuracy does not increase over 10 epochs, reduce the learning rate by half.
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, min_lr=0.00005, verbose=1)
batch_size = 128
history = model.fit(x=x_tr, y=y_tr, batch_size=16, epochs=100, verbose=2, shuffle=True, validation_data=(x_te, y_te), callbacks=[reduce_lr])


Train on 1024 samples, validate on 256 samples
Epoch 1/100
1024/1024 - 14s - loss: 5.0321 - accuracy: 0.0254 - val_loss: 4.7272 - val_accuracy: 0.0234
Epoch 2/100
1024/1024 - 12s - loss: 4.0504 - accuracy: 0.0762 - val_loss: 4.7831 - val_accuracy: 0.0273
Epoch 3/100
1024/1024 - 12s - loss: 3.5338 - accuracy: 0.1475 - val_loss: 4.6556 - val_accuracy: 0.0352
Epoch 4/100
1024/1024 - 12s - loss: 3.0477 - accuracy: 0.2832 - val_loss: 3.5732 - val_accuracy: 0.1680
Epoch 5/100
1024/1024 - 13s - loss: 2.6463 - accuracy: 0.3418 - val_loss: 3.3376 - val_accuracy: 0.2109
Epoch 6/100
1024/1024 - 13s - loss: 2.2836 - accuracy: 0.4570 - val_loss: 3.4909 - val_accuracy: 0.1914
Epoch 7/100
1024/1024 - 13s - loss: 2.0498 - accuracy: 0.5117 - val_loss: 2.7633 - val_accuracy: 0.2930
Epoch 8/100
1024/1024 - 13s - loss: 1.8321 - accuracy: 0.5615 - val_loss: 3.0966 - val_accuracy: 0.2891
Epoch 9/100
1024/1024 - 13s - loss: 1.6707 - accuracy: 0.6162 - val_loss: 2.3218 - val_accuracy: 0.4531
Epoch 10/100
1024

Epoch 76/100
1024/1024 - 12s - loss: 0.1201 - accuracy: 0.9766 - val_loss: 0.6646 - val_accuracy: 0.8516
Epoch 77/100
1024/1024 - 12s - loss: 0.1199 - accuracy: 0.9814 - val_loss: 0.6942 - val_accuracy: 0.8477
Epoch 78/100
1024/1024 - 12s - loss: 0.1188 - accuracy: 0.9766 - val_loss: 0.6811 - val_accuracy: 0.8477
Epoch 79/100
1024/1024 - 12s - loss: 0.1177 - accuracy: 0.9834 - val_loss: 0.6592 - val_accuracy: 0.8633
Epoch 80/100
1024/1024 - 12s - loss: 0.1221 - accuracy: 0.9746 - val_loss: 0.6838 - val_accuracy: 0.8438
Epoch 81/100
1024/1024 - 12s - loss: 0.1195 - accuracy: 0.9805 - val_loss: 0.6720 - val_accuracy: 0.8594
Epoch 82/100
1024/1024 - 13s - loss: 0.1191 - accuracy: 0.9795 - val_loss: 0.6723 - val_accuracy: 0.8516
Epoch 83/100
1024/1024 - 12s - loss: 0.1195 - accuracy: 0.9805 - val_loss: 0.6897 - val_accuracy: 0.8398
Epoch 84/100
1024/1024 - 12s - loss: 0.1168 - accuracy: 0.9814 - val_loss: 0.7038 - val_accuracy: 0.8438
Epoch 85/100
1024/1024 - 12s - loss: 0.1156 - accuracy:

In [17]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14, 6))
plt.rc('font', size=18)

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train_accuracy", "val_accuracy"])
plt.show()

<Figure size 1400x600 with 1 Axes>

In [38]:
import pickle
from glob import iglob
import numpy as np
import librosa
from shutil import rmtree
from constants import *


OUTPUT_VAL_DIR = '../output_val_una_re_augmentation'
OUTPUT_DIR_VAL_TRAIN = os.path.join(OUTPUT_VAL_DIR, 'train')
OUTPUT_DIR_VAL_TEST = os.path.join(OUTPUT_VAL_DIR, 'test')

DATA_AUDIO_VAL_DIR = '../vad_test_audio'

OUTPUT_DIR_RESAMPLED_VAL_TRAIN = os.path.join(OUTPUT_VAL_DIR, 'resampled_train')
OUTPUT_DIR_RESAMPLED_VAL_TEST = os.path.join(OUTPUT_VAL_DIR, 'resampled_test')

DATA_RESAMPLED_AUDIO_VAL_DIR = '../resampled_augmentation_val_data'

In [39]:
del_folder(DATA_RESAMPLED_AUDIO_VAL_DIR)
mkdir_p(DATA_RESAMPLED_AUDIO_VAL_DIR)

In [40]:
del_folder(OUTPUT_DIR_VAL_TRAIN)
del_folder(OUTPUT_DIR_VAL_TEST)
mkdir_p(OUTPUT_DIR_VAL_TRAIN)
mkdir_p(OUTPUT_DIR_VAL_TEST)

In [41]:
del_folder(OUTPUT_DIR_RESAMPLED_VAL_TRAIN)
del_folder(OUTPUT_DIR_RESAMPLED_VAL_TEST)
mkdir_p(OUTPUT_DIR_RESAMPLED_VAL_TRAIN)
mkdir_p(OUTPUT_DIR_RESAMPLED_VAL_TEST)

In [42]:
def resampling_val_audio():
     for i, wav_filename in enumerate(iglob(os.path.join(DATA_AUDIO_VAL_DIR, '**/**.wav'), recursive=True)):
        y, sr = librosa.load(wav_filename, sr=TARGET_SR, mono=True, duration=MAX_AUDIO_LENGTH)
        audio_duration = len(y) / sr
        
        resample_sr = TARGET_LENGTH / audio_duration        
        resample = librosa.resample(y, sr, resample_sr)
        print(i, wav_filename)
        print("Default SR : {}, Audio Length(Default SR) : {}, Audio Duration : {}".format(TARGET_SR, len(y), audio_duration))
        print("Audio Duration : {}, Resampling SR : {}, Result Audio Length : {}".format(audio_duration, resample_sr, len(resample)))
        
        if len(resample) < TARGET_LENGTH:
            resample = np.concatenate((resample, np.zeros(shape=(TARGET_LENGTH - len(resample), 1))))
        resample = resample[:TARGET_LENGTH]
        
        print("Final Audio Length : {}".format(len(resample)))

        
        output_folder = DATA_RESAMPLED_AUDIO_VAL_DIR
        if not os.path.exists(os.path.join(output_folder, wav_filename.split('/')[2])):
                mkdir_p(os.path.join(output_folder, wav_filename.split('/')[2]))
                
        output_filename = os.path.join(output_folder ,wav_filename.split('/')[2],str(wav_filename.replace('.','/').split('/')[5]) + '.pkl')
#         output_filename = os.path.join(output_folder, wav_filename.split('/')[2], str(wav_filename.split('/')[3][0]+".pkl"))
        print("Output File Name: {}\n".format(output_filename))
        
        result = {'resampled_audio': resample,
               'resample_sr': resample_sr}

        with open(output_filename, 'wb') as w:
                  pickle.dump(result, w)

In [43]:
resampling_val_audio()

0 ../vad_test_audio/2017019880001_kimsubin/5.wav
Default SR : 16000, Audio Length(Default SR) : 329874, Audio Duration : 20.617125
Audio Duration : 20.617125, Resampling SR : 9700.673590522441, Result Audio Length : 200000
Final Audio Length : 200000
Output File Name: ../resampled_augmentation_val_data/2017019880001_kimsubin/5.pkl

1 ../vad_test_audio/2017019880001_kimsubin/4.wav
Default SR : 16000, Audio Length(Default SR) : 346389, Audio Duration : 21.6493125
Audio Duration : 21.6493125, Resampling SR : 9238.168648542534, Result Audio Length : 200000
Final Audio Length : 200000
Output File Name: ../resampled_augmentation_val_data/2017019880001_kimsubin/4.pkl

2 ../vad_test_audio/2017019880001_kimsubin/6.wav
Default SR : 16000, Audio Length(Default SR) : 282013, Audio Duration : 17.6258125
Audio Duration : 17.6258125, Resampling SR : 11346.99464209097, Result Audio Length : 200001
Final Audio Length : 200000
Output File Name: ../resampled_augmentation_val_data/2017019880001_kimsubin/6

In [44]:
PAD_SIZE = 0
TARGET_LENGTH = 200000
TARGET_SR = 8000
MAX_AUDIO_LENGTH = 200
AUDIO_LENGTH = TARGET_LENGTH
SEGMENT_NUM = 1

In [45]:
def convert_val_data():

    for i, wav_filename in enumerate(iglob(os.path.join(DATA_RESAMPLED_AUDIO_VAL_DIR, '**/**.pkl'), recursive=True)):
        class_id = extract_class_id(wav_filename)
#         audio_buf = read_audio_from_filename(wav_filename, target_sr=TARGET_SR)
        audio_buf,_ = get_audio_buf(wav_filename)
        print(type(audio_buf))
        
        # normalize mean 0, variance 1
        audio_buf = (audio_buf - np.mean(audio_buf)) / np.std(audio_buf)
        original_length = len(audio_buf)
        print(i, wav_filename, original_length, np.round(np.mean(audio_buf), 4), np.std(audio_buf))
        
        voice_seg = []
        
        if original_length >= AUDIO_LENGTH + PAD_SIZE:
            audio_seg = audio_buf[PAD_SIZE : AUDIO_LENGTH + PAD_SIZE]
            voice_seg.append(audio_seg)
            print("Audio Segment Length : {}".format(len(audio_seg)))
        else:
            print('Drop Audio Segment, Audio length={}'.format(len(audio_buf)))
            
        output_folder = OUTPUT_DIR_RESAMPLED_VAL_TRAIN
            
        output_filename = os.path.join(output_folder ,wav_filename.split('/')[2]+"_"+str(wav_filename.replace('.','/').split('/')[5]) + '.pkl')
#         output_filename = os.path.join(output_folder, str(wav_filename[24:-4]).replace('/', '_') + '.pkl')

        out_segs = []
        for i_seg, audio_seg in enumerate(voice_seg) :
            out = {'class_id': class_id,
               'audio': audio_seg,
               'sr': TARGET_SR}
            
            with open(output_filename, 'wb') as w:
                pickle.dump(out, w)

            print("Output File Name : "+output_filename)
        
        print("-----------------")

if __name__ == '__main__':
    convert_val_data()

<class 'numpy.ndarray'>
0 ../resampled_augmentation_val_data/2017019880001_kimsubin/5.pkl 200000 0.0 0.99999994
Audio Segment Length : 200000
Output File Name : ../output_val_una_re_augmentation/resampled_train/2017019880001_kimsubin_5.pkl
-----------------
<class 'numpy.ndarray'>
1 ../resampled_augmentation_val_data/2017019880001_kimsubin/1.pkl 200000 -0.0 0.99999994
Audio Segment Length : 200000
Output File Name : ../output_val_una_re_augmentation/resampled_train/2017019880001_kimsubin_1.pkl
-----------------
<class 'numpy.ndarray'>
2 ../resampled_augmentation_val_data/2017019880001_kimsubin/4.pkl 200000 0.0 1.0
Audio Segment Length : 200000
Output File Name : ../output_val_una_re_augmentation/resampled_train/2017019880001_kimsubin_4.pkl
-----------------
<class 'numpy.ndarray'>
3 ../resampled_augmentation_val_data/2017019880001_kimsubin/6.pkl 200000 -0.0 1.0
Audio Segment Length : 200000
Output File Name : ../output_val_una_re_augmentation/resampled_train/2017019880001_kimsubin_6.pk

In [46]:
from sklearn.metrics import accuracy_score

val_files = glob(os.path.join(OUTPUT_DIR_RESAMPLED_VAL_TRAIN, '**.pkl'))
val_files.sort()
# print(val_files)
x_val, y_val = get_data(val_files)
x_val = x_val.reshape(-1, x_val.shape[1], 1)

print('y_val : ', y_val)
print(x_val.shape, y_val.shape)


y_val :  [ 0  0  0  0  0  0 13 13 13 13 13 13 28 43 43 43 43 43 43 83 83 83 83 83
 83]
(25, 200000, 1) (25,)


In [47]:
pred_out = model.predict(x_val)

real =[]
pred_out_idex=[]

list_dir = os.listdir(DATA_RESAMPLED_AUDIO_DIR)
list_dir.sort()

j = 0
for i,pred in enumerate(pred_out) :
    if np.argmax(pred) != y_val[i]:
        print('\033[31m'+"Predict :"+str(list_dir[np.argmax(pred)])+" , Real :"+str(list_dir[y_val[i]])+", Likelihood :"+str(np.max(pred)))
    else:
        print('\033[30m'+"Predict :"+str(list_dir[np.argmax(pred)])+" , Real :"+str(list_dir[y_val[i]])+", Likelihood :"+str(np.max(pred)))
    real.append(list_dir.index(list_dir[y_val[i]]))
    pred_out_idex.append(np.argmax(pred))
    
# get the accuracy
print ('\n\033[30m'+"Final Predict Likelihood : "+format(accuracy_score(real, pred_out_idex)*100, ".4f"))

Predict :2017019740001_kwakjuheon , Real :2017019740001_kwakjuheon, Likelihood :0.99424756
Predict :2017019740001_kwakjuheon , Real :2017019740001_kwakjuheon, Likelihood :0.99950683
Predict :2017019740001_kwakjuheon , Real :2017019740001_kwakjuheon, Likelihood :0.9981945
Predict :2017019740001_kwakjuheon , Real :2017019740001_kwakjuheon, Likelihood :0.97704405
Predict :2017019740001_kwakjuheon , Real :2017019740001_kwakjuheon, Likelihood :0.9876264
Predict :2017019740002_kimjiho , Real :2017019740001_kwakjuheon, Likelihood :0.9678687
Predict :2017019740014_leesumin , Real :2017019740014_leesumin, Likelihood :0.7057984
Predict :2017019740038_choihyeonseok , Real :2017019740014_leesumin, Likelihood :0.5008034
Predict :2017019740038_choihyeonseok , Real :2017019740014_leesumin, Likelihood :0.57155854
Predict :2017019740014_leesumin , Real :2017019740014_leesumin, Likelihood :0.62993175
Predict :2017019740014_leesumin , Real :2017019740014_leesumin, Likelihood :0.53090787
Predict :20170197